In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd '/content/drive/MyDrive/aiConnect_3rd/'
!pwd

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/aiConnect_3rd
/content/drive/MyDrive/aiConnect_3rd


In [ ]:
import os
cwd = '/content/drive/MyDrive/aiConnect_3rd/data'

In [ ]:
!pip install transformers[sentencepiece]

In [ ]:
!pip install "sacrebleu[ko]"

In [ ]:
import pandas as pd

train_df = pd.read_csv(os.path.join(cwd,'train.csv'))
train_df.head()

,id,text,mt,target
0,FAQEQ1Rt1ncRSxO,The winner of the first season of the show Gol...,쇼의 첫 번째 시즌의 우승자인 Golos Dina Garipova는 여배우 Irin...,쇼의 첫 번째 시즌의 우승자인 골로스디나 가리포바는 여배우 이리나 무라베바에게 헌정...
1,JSjY2hYQT4oPdz6,"Replica census entry for Gypsie Runningcloud, ...",Gypsie Runningcloud에 대한 복제 인구 조사 항목(48세).,"집시 런닝클라우드의 인구조사 자료 사본, 48세."
2,nykRvvrs3DjlWTQ,I don't think I've ever seen one where they in...,나는 그들이 대화를 포함하는 것을 본 적이 없다고 생각합니다.,저는 그것들이 대화를 포함하는 것을 본 적이 없다고 생각합니다.
3,yIzKFCfNBhLx2q5,The patients were not significantly slower on ...,"환자들은 회상, 반응 시간 또는 추론 테스트에서 유의하게 느려지지 않았습니다.","환자들은 회상, 반응 시간 또는 추론 테스트에서 유의하게 느려지지 않았습니다."
4,68Yvljqe5BS5CWQ,"The artist declares, despite her outward fragi...",그 예술가는 겉으로는 연약함에도 불구하고 엄격한 지도자라고 선언한다.,그 예술가는 겉으로는 연약함에도 불구하고 엄격한 지도자라고 선언한다.


# **HuggingFace Pipeline**

In [ ]:
from transformers import pipeline
from sacrebleu.metrics import BLEU

In [23]:
pipe = pipeline(task='text2text-generation', model='facebook/m2m100_418M', device='cuda')

pred = pipe(list(train_df.loc[:5,'text']), forced_bos_token_id=pipe.tokenizer.get_lang_id(lang='ko'))

print('original text:', train_df.loc[0,'text'])
print('target:',train_df.loc[0,'target'])
print('prediction:',pred[0]['generated_text'])

bleu = BLEU(trg_lang='ko')
print(f"BLUE score: {bleu.corpus_score([r['generated_text'] for r in pred], [list(train_df.loc[:5,'target'])]).score}")

original text: The winner of the first season of the show GolosDina Garipova sings a song from the film Carnival in a program dedicated to actress Irina Murav'eva.
target: 쇼의 첫 번째 시즌의 우승자인 골로스디나 가리포바는 여배우 이리나 무라베바에게 헌정된 프로그램에서 영화 카니발의 노래를 부릅니다.
prediction: 쇼의 첫 번째 시즌의 우승자 GolosDina Garipova는 영화 카니발에서 여배우 Irina Murav'eva에 전념하는 프로그램에서 노래를 부릅니다.
BLUE score: 26.35921813968538


In [ ]:
pipe = pipeline("translation", model="sungmogi/en2ko_hiphop_small-100", device='cuda', src_lang='en', tgt_lang='ko', max_length=512)

pred = pipe(list(train_df.loc[:5,'text']))
print('original text:', train_df.loc[0,'text'])
print('target:',train_df.loc[0,'target'])
print('prediction:',pred[0]['translation_text'])

bleu = BLEU(trg_lang='ko')
print(f"BLUE score: {bleu.corpus_score([r['translation_text'] for r in pred], [list(train_df.loc[:5,'target'])]).score}")

original text: The winner of the first season of the show GolosDina Garipova sings a song from the film Carnival in a program dedicated to actress Irina Murav'eva.
target: 쇼의 첫 번째 시즌의 우승자인 골로스디나 가리포바는 여배우 이리나 무라베바에게 헌정된 프로그램에서 영화 카니발의 노래를 부릅니다.
prediction: 쇼의 첫 시즌의 수상자, GolosDina Garipova는 여배우 Irina Murav'ev에게 바치는 프로그램에서 영화 Carnival에서 노래를 불러
BLUE score: 9.793252905434562


In [18]:
pipe = pipeline("translation", model="KETI-AIR-Downstream/long-ke-t5-base-translation-aihub-en2ko", device='cuda', src_lang='en', tgt_lang='ko', max_length=512)

pred = pipe(list(train_df.loc[:5,'text']))
print('original text:', train_df.loc[0,'text'])
print('target:',train_df.loc[0,'target'])
print('prediction:',pred[0]['translation_text'])

bleu = BLEU(trg_lang='ko')
print(f"BLUE score: {bleu.corpus_score([r['translation_text'] for r in pred], [list(train_df.loc[:5,'target'])]).score}")

original text: The winner of the first season of the show GolosDina Garipova sings a song from the film Carnival in a program dedicated to actress Irina Murav'eva.
target: 쇼의 첫 번째 시즌의 우승자인 골로스디나 가리포바는 여배우 이리나 무라베바에게 헌정된 프로그램에서 영화 카니발의 노래를 부릅니다.
prediction: 이번 쇼의 첫 시즌 우승자 골로스디나 가리포바는 여배우 이리나 무라브'에바에게 바치는 프로그램에서 영화 카니발 노래를 부르며, 올로스디나 가리포바는 여배우 올로스디나 가리포바의 1기 시즌 우승자가 영화 '카니발'의 노래를 부른다.
BLUE score: 21.035823575458114


In [ ]:
pipe = pipeline("translation", model="NHNDQ/nllb-finetuned-en2ko", device='cuda', src_lang="en", tgt_lang="ko", max_length=512)

pred = pipe(list(train_df.loc[:5,'text']))
print('original text:', train_df.loc[0,'text'])
print('target:', train_df.loc[0,'target'])
print('prediction:', pred[0]['translation_text'])

bleu = BLEU(trg_lang='ko')
print(f"BLUE score: {bleu.corpus_score([r['translation_text'] for r in pred], [list(train_df.loc[:5,'target'])]).score}")

original text: The winner of the first season of the show GolosDina Garipova sings a song from the film Carnival in a program dedicated to actress Irina Murav'eva.
target: 쇼의 첫 번째 시즌의 우승자인 골로스디나 가리포바는 여배우 이리나 무라베바에게 헌정된 프로그램에서 영화 카니발의 노래를 부릅니다.
prediction: ko1 시즌 우승자 GolosDina Garipova는 영화 카니발의 노래를 여배우 Irina Murav'eva를 위한 프로그램에서 부른다.
BLUE score: 36.8210818605552


In [20]:
pipe = pipeline("translation", model="NHNDQ/nllb-finetuned-en2ko", device='cuda', src_lang="en", tgt_lang="kor_Hang", max_length=512)

pred = pipe(list(train_df.loc[:5,'text']))
print('original text:', train_df.loc[0,'text'])
print('target:', train_df.loc[0,'target'])
print('prediction:', pred[0]['translation_text'])

bleu = BLEU(trg_lang='ko')
print(f"BLUE score: {bleu.corpus_score([r['translation_text'] for r in pred], [list(train_df.loc[:5,'target'])]).score}")

original text: The winner of the first season of the show GolosDina Garipova sings a song from the film Carnival in a program dedicated to actress Irina Murav'eva.
target: 쇼의 첫 번째 시즌의 우승자인 골로스디나 가리포바는 여배우 이리나 무라베바에게 헌정된 프로그램에서 영화 카니발의 노래를 부릅니다.
prediction: 쇼 첫 시즌의 우승자 골로스디나 가리포바는 영화 카니발의 노래를 여배우 이리나 무라브에바를 위한 프로그램에서 부른다.
BLUE score: 50.146331652329806


# **파이프라인으로 추론**

In [24]:
# 가비지 컬렉션
import gc
import torch

gc.collect()
torch.cuda.empty_cache()

In [ ]:
from torch.utils.data import Dataset
from tqdm import tqdm

test_df = pd.read_csv(os.path.join(cwd,'test.csv'))
submission_df = pd.read_csv(os.path.join(cwd,'sample_submission.csv'))

class TestDataset(Dataset):
  def __init__(self, text):
    self.lst = list(text)

  def __len__(self):
    return len(self.lst)

  def __getitem__(self,i):
    return self.lst[i]

test_dataset = TestDataset(test_df['text'])

preds = [i for i in tqdm(pipe(test_dataset, max_length=512))] # 여기 끝나면 preds 찍어볼것. 옵션으로  max_length=512 추가.

100%|██████████| 10757/10757 [1:05:30<00:00,  2.74it/s]


In [ ]:
# preds에서 "translation_text" 키의 값을 추출하여 결과 리스트에 저장
submission_df['pred'] = [item[0]['translation_text'] for item in preds]

submission_df.tail()

,id,pred
10752,FomIsSfNWQN08Dp,모든 k-popen 그룹은 가사에 [두근] 또는 [두근-두근-대]가 포함된 노래를 ...
10753,vjR4nFxkKeGaZNU,아시아인에 대한 인종차별이 심각한 문제라는 것을 다시금 상기시켰다.
10754,sXs4MznOw8Qobb5,연구팀은 아나멘시스와 아파렌시스의 식단과 생활 방식이 어떻게 달랐을지 보다 자세히 ...
10755,SEjj7LbLKUTwhrZ,관객의 캘리포니아인들은 바로 그 쪽에 투표할 것이다.
10756,AFMWCvZ0zFo7Ge9,"하지만 우리가 점점 커지고 커지면서, 나는 그것에 대해 약간 무감각해졌다고 생각한다."


In [ ]:
submission_df.to_csv(os.path.join(cwd,'prediction_5th.csv'), index=False)